## SOURCE TO BRONZE LAYER

### Process:

> The function fetches daily stock data using the Yahoo Finance API (`yfinance`).


In [1]:
# Import necessary libraries
import pandas as pd
from datetime import  timedelta
import yfinance as yf
from requests import Session
from pyrate_limiter import Limiter, Duration, RequestRate
from requests_cache import CacheMixin, SQLiteCache
from requests_ratelimiter import LimiterMixin, MemoryQueueBucket
from ETLTools import DateTimeUtil, GlobalPath

In [2]:
# Importing Common Utility Funcation
%run ../COMMON/common_utility.ipynb

In [3]:
# Define Constants file paths
HOLDINGS_HISTORY_PATH = GlobalPath("DATA/SOURCE/Holdings/HoldingsHistory_data.csv")
STOCKDATA_BRONZE_LAYER_PATH = GlobalPath("DATA/BRONZE/StockData")

In [4]:
# Setting cache location for yfinance
yf.set_tz_cache_location(".cache")

# Rate limiting setup
class CachedLimiterSession(CacheMixin, LimiterMixin, Session):
    pass

history_rate = RequestRate(1, Duration.SECOND * 2)
limiter = Limiter(history_rate)
session = CachedLimiterSession(
    limiter=limiter,
    bucket_class=MemoryQueueBucket,
    backend=SQLiteCache(".cache/session", expire_after=timedelta(hours=1)),
)

In [5]:
def process_file(stock_ticker, date, output_file):
    """Fetch historical stock data and save it to a CSV file."""
    df = stock_ticker.history(
        start=date.start_date,
        end=min(date.end_date, DateTimeUtil.today()),
        interval="1d",
        actions=True,
        rounding=True,
    )
    if df.empty:
        raise Exception(f"No data fetched for : from {date.start_date} to {date.end_date}")
    df = df.reset_index()
    df = replace_punctuation_from_columns(df)
    df = fix_duplicate_column_names(df)
    df = df.round(2)
    df.to_csv(output_file, index=False)
    print(f"Data processed and saved to: {output_file}")

In [6]:
# Dictionary for stock ticker overrides
OVERWRITE_TICKERS = {
    "BAJAJHFL":"BAJAJHFL.NS",
    "BHAGERIA": "BHAGERIA.NS",
    "BPCL": "BPCL.NS",
    "GOLDBEES": "GOLDBEES.NS",
    "HERANBA": "HERANBA.NS",
    "IDEA": "IDEA.NS",
    "INFY": "INFY.NS",
    "IRCTC": "IRCTC.NS",
    "KPITTECH": "KPITTECH.NS",
    "LICI": "LICI.NS",
    "MIRAE_ASSET_ELSS_TAX_SAVER_FUND_DIRECT_PLAN_GROWTH": "0P00017844.BO",
    "NIFTYBEES": "NIFTYBEES.NS",
    "PNB": "PNB.NS",
    "SBIN": "SBIN.NS",
    "SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH": "0P0000XVL9.BO",
    "TATACHEM": "TATACHEM.NS",
    "TATAMOTORS": "TATAMOTORS.NS",
    "TATAPOWER": "TATAPOWER.NS",
    "VOLTAS": "VOLTAS.NS",
    "YESBANK": "YESBANK.NS",
}

In [7]:
# Load holdings data from CSV
df_holdings_history = pd.read_csv(HOLDINGS_HISTORY_PATH)
df_holdings_history["min_date"] = pd.to_datetime(df_holdings_history["min_date"])
df_holdings_history["max_date"] = pd.to_datetime(df_holdings_history["max_date"])
print(f"Loaded data from: {HOLDINGS_HISTORY_PATH}")

Loaded data from: /workspaces/PortfolioTracker/DATA/SOURCE/Holdings/HoldingsHistory_data.csv


In [8]:
# Fetch stock data and process it.
for _, row in df_holdings_history.iterrows():
  print(f"\nProcessing data for symbol {row['symbol']}:")
  try:
    stock_ticker = yf.Ticker(
        OVERWRITE_TICKERS.get(row["symbol"], row["isin"]),
        session=session,
    )
    date_list = generate_date_list(row["min_date"].to_pydatetime(), row["max_date"].to_pydatetime())   
    for date in date_list:
        output_file = STOCKDATA_BRONZE_LAYER_PATH.joinpath(f"{row['symbol']}_{date.year:04d}_{date.month:02d}.csv")
        if output_file.exists() and date.month_difference(DateTimeUtil.today()) >= 1:
            continue
        process_file(stock_ticker, date, output_file)
  except Exception as e:
    print(f"Error processing {row['symbol']}:\n{e}")


Processing data for symbol BAJAJHFL:


BAJAJHFL.NS: Period '1mo' is invalid, must be one of ['1d', '5d', 'ytd', 'max']


Error processing BAJAJHFL:
No data fetched for : from 2024-09-01 00:00:00 to 2024-09-30 00:00:00

Processing data for symbol BHAGERIA:


Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/BHAGERIA_2020_05.csv


Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/BHAGERIA_2020_06.csv

Processing data for symbol BPCL:


...

Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2024_05.csv


Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2024_06.csv


Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2024_07.csv


Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2024_08.csv


Data processed and saved to: /workspaces/PortfolioTracker/DATA/BRONZE/StockData/SBI_LONG_TERM_EQUITY_FUND_DIRECT_PLAN_GROWTH_2024_09.csv
